# Compact version of explicit scheme
Just the same as before, only now with less output
* adding "getC" to the definitons

## Definitions


In [73]:
import openmesh as om
import numpy as np
import matplotlib.pyplot as plt

mesh = om.read_trimesh('T4.off')
V=mesh.points()
X=V[:,0]
Y=V[:,1]

def initActiveNodes():
    ActiveNodeList = []
    for vh in mesh.vertices():
        k = vh.idx()
        if V[k,0] == 0 or V[k,1] == 1:
            ActiveNodeList.append(vh)
    return ActiveNodeList

def initActiveFaceList():
    ActiveFaceList = []

    i=0
    for fh in mesh.faces():
        counter = 0
    
        for vh in mesh.fv(fh):
            if vh in ActiveNodeList:
                # print(vh.idx(),' in ', fh.idx())                
                counter = counter + 1
            
        if counter == 2:
            ActiveFaceList.append(fh)

    return ActiveFaceList

def init_TS_nodes():
    T = np.zeros(len(V))+7
    S = np.zeros(len(V))+7

    for vh in ActiveNodeList:    
        k = vh.idx()
        if V[k,0] == 0:
            T[k] = 0
            S[k] = 0
        elif V[k,1] == 1:
            T[k] = V[k,0]
            S[k] = 0
    
    return T, S

def init_TS_faces():
    Tf = np.zeros(len(mesh.faces()))+7
    Sf = np.zeros(len(mesh.faces()))+7

    for fh in ActiveFaceList:
        T_ABC = 0
        S_ABC = 0
        counter = 0
        for vh in mesh.fv(fh):
            if vh in ActiveNodeList:
                T_ABC = T_ABC + T[vh.idx()]
                S_ABC = S_ABC + S[vh.idx()]            
                counter = counter + 1
            
        if counter != 2:
            print('ERR at face', fh.idx())
        else:
            Tf[fh.idx()] = T_ABC / 2
            Sf[fh.idx()] = S_ABC / 2
            
    return Tf, Sf
    
# routine to get active face indices    
def getActiveFaceIndices():
    I_list = []
    for fh in ActiveFaceList:
        I_list.append(fh.idx())
        
    return I_list

    
def getActiveNodeIndices():
    I_list = []
    for fh in ActiveNodeList:
        I_list.append(fh.idx())

    return I_list


def getProcessedFaceIndices():
    I_list = []
    for fh in ProcessedFaceList:
        I_list.append(fh.idx())
        
    return I_list

def getObsoleteFaceIndices():
    I_list = []
    for fh in ObsoleteFaceList:
        I_list.append(fh.idx())
        
    return I_list

def get_T_C_dummy(T_A, T_B):
    return (T_A + T_B)/2 + 10

# ... update faces: don inline    

def getAngle(v1, v2):
    angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))    
    return angle


def getC(I):
    A = V[I[0],0:2]
    B = V[I[1],0:2]
    C = V[I[2],0:2]    
    TA = T[I[0]]
    TB = T[I[1]]
    

    a = sum((B-C)**2)**(1/2)
    b = sum((A-C)**2)**(1/2)
    c = sum((A-B)**2)**(1/2)
    
    # firstround = 1
    TC=min(TA,TB)
    TC=(TA+TB)/2
    
    p1 = abs(TC-TA)/b
    p2 = abs(TC-TB)/a
    p3 = abs(TA-TB)/c
    
    alpha = getAngle(B-A, C-A)
    beta = getAngle(A-B, C-B)
    gamma = getAngle(A-C, B-C)

    asum=alpha+beta+gamma
    if asum != np.pi:
        print('ERR: angles do not sum up to pi: %.2f' % asum)
      

    YC = V[I[2],1] # C[1]
    SC = (TA+TB+1e-5)/2    
    fC = SC / (YC+1e-5)
    
    tfrac=abs(TA-TB)/(c*fC+1e-5)
    if tfrac > 0.99:
        print('ERR: tfrac greater 1: %.2f' %tfrac)
        theta = np.pi
    else:
        theta = np.arcsin(tfrac)
    
    counter = 0
    if p2 < fC:
        counter = counter + 1
    else:
        print('ERR: Huygens principle')

    if theta < beta:
        counter = counter + 1
    else:
        print('ERR: Evaluation order')
        
    if theta + alpha < np.pi / 2:
        counter = counter + 1
    else:    
        print('ERR: Causality: theta: %.2f' %theta, ', alpha: %.2f' % alpha)        

    if counter == 3:
        HC = 1/np.sin(gamma)*np.sqrt(p1**2 - 2*p1*p2*np.cos(gamma)+p2**2)
    else:
        HC = min(TC, TA+b*fC, TB +a*fC)
    
##########################################################################################################    
#    HC = 1/np.sin(gamma)*np.sqrt(p1**2 - 2*p1*p2*np.cos(gamma)+p2**2)
    
    TC = HC
        
    return TC

I = [0, 5, 30]
TC = getC(I)

ActiveNodeList = initActiveNodes()
ActiveFaceList = initActiveFaceList()
    
T, S = init_TS_nodes()
Tf, Sf = init_TS_faces()

I_active_faces = getActiveFaceIndices()
print('active faces:', I_active_faces)


active faces: [1, 4, 5, 8, 11, 22, 40, 74, 75]


## Running

In [74]:
ProcessedFaceList = []
ObsoleteFaceList = []

ActiveNodeList = initActiveNodes()
ActiveFaceList = initActiveFaceList()
global_I = I

# print('len: list(I[K]) \t \t (id_A, id_B) -> id_C \n \n \n')
j=0
while len(ActiveFaceList)>0:
    j=j+1
    
    # (0) update the index
    I_active_faces = getActiveFaceIndices()

    I = np.array(I_active_faces)
    K = np.argsort(Tf[I], axis=0)

    # print('(len)', len(I_active_faces),': (f)', list(I[K]))
    
    # (1) obtain id of next face
    next_face_id = (I[K])[0]
    fh = mesh.face_handle(next_face_id)
    
    # quick check whether the face is obsolete,
    # which is a normal situation that should not be handled as an error
    # but by moving on to the next face

    counter = 0
    for vh in mesh.fv(fh):
        if vh in ActiveNodeList:
            counter = counter + 1    
    if counter == 3:
        # print('remove obsolete face:', fh.idx())
        ObsoleteFaceList.append(fh)
        ActiveFaceList.remove(fh)
        continue
            
    # (2) calculate value of outgoing vertex
    counter = 0
    for vh in mesh.fv(fh):
        if vh in ActiveNodeList:
            # print(vh.idx(),'ingoing')
            if counter == 0:
                counter = counter + 1
                id_A = vh.idx()
                T_A = T[vh.idx()]                
            elif counter == 1:
                counter = counter + 1
                id_B = vh.idx()
                T_B = T[vh.idx()]
            else:
                print('ERR: too many ingoing nodes')
                
        else:
            # print(vh.idx(),'outgoing')        
            id_C = vh.idx()
            vh_C = vh

######################################################################################################

    if counter == 2:
        I_ABC = [id_A, id_B, id_C]
        # I = np.array(I)
        global_I = I_ABC

        # T_C_dummy = get_T_C_dummy(T_A, T_B)        
        T_C = getC(I_ABC)
        
        # print('T', I_ABC,' : %.2f' %T_C)        
        # T_C = T_C_dummy        
        # T_C = T_C + j / 1000
        
        T[id_C] = T_C
    else:
        print('ERR: missing inputs for calculating T_C')
        # T[id_C] = j
        
    # T[id_C] = j
    
    # (3) handle the list of active faces
    if (not vh_C in ActiveNodeList):
        ActiveNodeList.append(vh_C) # also ! 
    else:
        print('node already listed:', vh_C.idx())
    
    ActiveFaceList.remove(fh)
    ProcessedFaceList.append(fh)

    # (3b) agregate new faces to the list
    f_list = []
    print('(', id_A, ',', id_B, ') ->', id_C,'\t \t',len(I_active_faces),': ', list(I[K]))    
    for fvh in mesh.vf(vh_C):
        f_list.append(fvh.idx())
        # print('fv', fvh.idx())
        if (not fvh in ActiveFaceList) and (not fvh in ProcessedFaceList):
            counter = 0
            for vh in mesh.fv(fvh):
                if vh in ActiveNodeList:
                    counter = counter + 1
        
            if counter == 2:
                ActiveFaceList.append(fvh)
                # print('new face:', fvh.idx())
                
                counter = 0
                for vh in mesh.fv(fvh):
                    if vh in ActiveNodeList:
                        if counter == 0:
                            T_A = T[vh.idx()]
                            counter = counter + 1
                        elif counter == 1:
                            T_B = T[vh.idx()]
                    Tf[fvh.idx()] = (T_A + T_B)/2
            
            elif counter == 3:
                print('face duplicate registered:', fvh.idx())                
        
print('\n T: \n', T)
print('\n Tf: \n', Tf)

( 5 , 0 ) -> 30 	 	 9 :  [5, 8, 11, 74, 75, 1, 40, 22, 4]
( 5 , 10 ) -> 34 	 	 10 :  [8, 11, 74, 75, 0, 9, 1, 40, 22, 4]
( 15 , 10 ) -> 38 	 	 11 :  [11, 74, 75, 0, 9, 33, 7, 1, 40, 22, 4]
( 20 , 25 ) -> 46 	 	 12 :  [74, 75, 0, 9, 33, 7, 6, 43, 1, 40, 22, 4]
( 20 , 15 ) -> 42 	 	 13 :  [75, 0, 9, 33, 7, 6, 43, 76, 1, 35, 40, 22, 4]
ERR: Causality: theta: 0.00 , alpha: 2.75
( 30 , 0 ) -> 1 	 	 14 :  [0, 9, 33, 7, 6, 43, 76, 44, 73, 1, 35, 40, 22, 4]
( 34 , 30 ) -> 6 	 	 13 :  [33, 7, 6, 43, 76, 44, 73, 32, 1, 35, 40, 22, 4]
( 38 , 34 ) -> 11 	 	 13 :  [6, 43, 76, 44, 73, 32, 34, 31, 1, 35, 40, 22, 4]
( 42 , 46 ) -> 21 	 	 12 :  [44, 73, 32, 34, 31, 71, 30, 1, 35, 40, 22, 4]
( 42 , 38 ) -> 16 	 	 13 :  [73, 32, 34, 31, 71, 30, 72, 1, 35, 42, 40, 22, 4]
( 6 , 1 ) -> 31 	 	 12 :  [31, 71, 30, 72, 68, 70, 1, 35, 42, 40, 22, 4]
( 6 , 11 ) -> 35 	 	 12 :  [30, 72, 68, 70, 29, 10, 1, 35, 42, 40, 22, 4]
( 21 , 16 ) -> 43 	 	 12 :  [68, 70, 29, 10, 15, 36, 1, 35, 42, 40, 22, 4]
( 16 , 11 ) -> 3

## Checksum

In [51]:
I_active_nodes = getActiveNodeIndices()
print('active nodes:', I_active_nodes)


active nodes: [0, 5, 10, 15, 20, 25, 26, 27, 28, 29, 30, 34, 38, 46, 42, 1, 6, 11, 21, 16, 31, 35, 43, 39, 2, 7, 47, 17, 12, 32, 22, 36, 40, 3, 44, 8, 13, 48, 18, 33, 37, 23, 41, 45, 19, 24, 4, 9, 14, 49]


In [52]:

I_active_nodes = getActiveNodeIndices()
print('active nodes (usorted):', I_active_nodes)
print(len(I_active_nodes),': ',np.sort(I_active_nodes))

I_faces = getProcessedFaceIndices()
I_faces_obsolete = getObsoleteFaceIndices()
print('check completeness:', len(I_faces), len(I_faces_obsolete))
print('faces, processed:', np.sort(I_faces))
print('faces, obsolete:', np.sort(I_faces_obsolete))


active nodes (usorted): [0, 5, 10, 15, 20, 25, 26, 27, 28, 29, 30, 34, 38, 46, 42, 1, 6, 11, 21, 16, 31, 35, 43, 39, 2, 7, 47, 17, 12, 32, 22, 36, 40, 3, 44, 8, 13, 48, 18, 33, 37, 23, 41, 45, 19, 24, 4, 9, 14, 49]
50 :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
check completeness: 40 40
faces, processed: [ 0  2  3  5  6  8 10 11 12 13 15 18 20 21 24 26 27 30 31 33 37 38 41 44
 49 50 51 54 56 59 62 64 65 68 70 73 74 75 77 79]
faces, obsolete: [ 1  4  7  9 14 16 17 19 22 23 25 28 29 32 34 35 36 39 40 42 43 45 46 47
 48 52 53 55 57 58 60 61 63 66 67 69 71 72 76 78]
